Use Azure OpenAI to query the files.

In [ ]:
#!pip install pymupdf
#!pip install python-docx
#!pip install python-dotenv
#%pip install azure-openai
#%pip install openai

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement azure-openai (from versions: none)
ERROR: No matching distribution found for azure-openai


In [1]:
import fitz  
import json
import os
import openai
from docx import Document
from dotenv import load_dotenv
from openai import AzureOpenAI

In [7]:
load_dotenv("azure.env")

api_type: str = "azure"
api_key = os.getenv("OPENAI_API_KEY")
api_base = os.getenv("OPENAI_API_BASE")

model = "gpt-4o"

In [8]:
print(f"OpenAI version: {openai.__version__}")
print(api_base)
print(model)

OpenAI version: 1.54.3
https://ais-aoai-wrb-regulatory-poc.openai.azure.com/
gpt-4o


In [9]:
def read_file(file_path):
    file_extension = os.path.splitext(file_path)[1].lower()
    
    if file_extension == '.pdf':
        try:
            text = ""
            pdf_document = fitz.open(file_path)
            for page_num in range(pdf_document.page_count):
                page = pdf_document.load_page(page_num)
                text += page.get_text("text") 
            return text
        except Exception as e:
            print(f"[ERROR] Could not read the PDF file {file_path}: {e}")
            return None
    
    elif file_extension == '.txt':
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                return f.read()
        except Exception as e:
            print(f"[ERROR] Could not read the txt file {file_path}: {e}")
            return None
    
    elif file_extension == '.json':
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
                return json.dumps(data, indent=2)
        except Exception as e:
            print(f"[ERROR] Could not read the JSON file {file_path}: {e}")
            return None

    elif file_extension == '.docx':
        try:
            text = ""
            doc = Document(file_path)
            for para in doc.paragraphs:
                text += para.text + "\n" 
            return text
        except Exception as e:
            print(f"[ERROR] Could not read the docx file {file_path}: {e}")
            return None
    
    else:
        print(f"[ERROR] Unsupported file type: {file_extension}")
        return None

In [10]:
def gpt_file_analysis(file_content, prompt):
    client = AzureOpenAI(
        api_key=api_key,
        api_version="2024-07-01-preview",
        base_url=f"{api_base}/openai/deployments/{model}",
    )

    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": (
                    "You are a helpful assistant that analyzes text content from PDF, TXT, JSON, and DOCX files. "
                    "When generating responses, prioritize providing clear and concise information directly related to the file's content. "
                    "Follow these rules strictly in your responses: "
                    "- If specific information is available, include it in your response with emphasis on clarity and brevity. "
                    "- If the file cannot be read or the information cannot be extracted, please specify the reason."
                ),
            },
            {
                "role": "user",
                "content": f"{prompt}\n\n{file_content}",
            },
        ],
        max_tokens=2000,
        temperature=0.3,
    )

    if response.choices:
        return response.choices[0].message.content
    else:
        return "Error: No response from model."


In [ ]:
file_path = "ISO CSP Manual Coding and Rules.pdf"
file_content = read_file(file_path)
if file_content is None:
    print("[ERROR] Could not read the file. Please check the path and format.")
    exit()

prompt = "Briefly summarize the content of the document. In pagraph form, describe the main topics covered in the document and any key points or findings."
result = gpt_file_analysis(file_content, prompt)

print("[INFO] Analysis completed.")
print(result)

[INFO] Analysis completed.
The document is a comprehensive guide for coding and reporting general liability insurance data, specifically focusing on unmanned aircraft (drones) and cyber insurance. It includes detailed instructions and codes for various fields such as accounting date, aggregate policy limits, annual statement line of business codes, classification codes, company numbers, coverage codes, deductible amounts, and more. The document also provides specific codes for different types of losses, policy limits, and transaction types. Additionally, it outlines the requirements for reporting premiums, exposures, and losses, including special instructions for cancellations and exceptions. The guide is intended for use by insurance companies to ensure accurate and consistent data submission to the Insurance Services Office (ISO).


In [ ]:
file_path = "ISO CSP Manual Coding and Rules.pdf"
file_content = read_file(file_path)
if file_content is None:
    print("[ERROR] Could not read the file. Please check the path and format.")
    exit()

prompt = "what are the valid code and values for ISO CSP reporting for each attribute? I want in a table format"
result = gpt_file_analysis(file_content, prompt)

print("[INFO] Analysis completed.")
print(result)